## Prepare the Base Model

In [1]:
import os

In [2]:
%pwd

'd:\\iNeuron_Projects\\cnnClassifier\\cnnClassifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\iNeuron_Projects\\cnnClassifier\\cnnClassifier'

### make data class:

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:         # For check items below check a params.yaml and config.yaml
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

### configuration manager

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

### Class ConfigurationManager

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(           # For check items below check a params.yaml and config.yaml.
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

### Import additional libraries

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2023-11-08 15:37:18,102: INFO: utils: NumExpr defaulting to 4 threads.]


### Define a PrepareBaseModel class

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    # Method to obtain the base model (VGG16)
    def get_base_model(self):                                  
        self.model = tf.keras.applications.vgg16.VGG16(                       # Create the VGG16 base model with specified input shape, weights, and top layer inclusion
            input_shape=self.config.params_image_size,                        # For check items below check a params.yaml and config.yaml.
            weights=self.config.params_weights,
            include_top=self.config.params_include_top                   
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)   # Check a config.yaml

    
    @staticmethod                                                             # used to prepare the full model for the convolutional neural network (CNN).
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate): 
        if freeze_all:                                                        # Depending on the freeze settings, freeze certain layers of the base model
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)                   # Flatten Layer
        prediction = tf.keras.layers.Dense(                                    # Dense Layer (tf.keras.layers.Dense 
            units=classes,
            activation="softmax"                                               # Softmax activation for multi-class classification
        )(flatten_in)

        full_model = tf.keras.models.Model(                                    # Create the full model with the custom output layer
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),    # SGD optimizer with specified learning rate.
            loss=tf.keras.losses.CategoricalCrossentropy(),                    # Categorical cross-entropy loss for classification
            metrics=["accuracy"]                                               # Accuracy metric for evaluation
        )                                              

        full_model.summary()                                                   # Display a summary of the full model's architecture
        return full_model
        


    def update_base_model(self):                                               # for update_base_model
        self.full_model = self._prepare_full_model(                            # Prepare the full model by calling the static method with specific parameters
            model=self.model,
            classes=self.config.params_classes,                                # Number of output classes from the configuration
            freeze_all=True,                                                   # Freeze all layers except the custom output layer
            freeze_till=None,                                                  # No specific layers are frozen in this case
            learning_rate=self.config.params_learning_rate                     # Learning rate from the configuration
        )
        # Save the updated full model to a specified path   
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    # Static method to save a Keras model to a specified path.
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model): 
        model.save(path)

### Test a model :

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-11-08 15:40:44,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-08 15:40:44,086: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-08 15:40:44,088: INFO: common: created directory at: artifacts]
[2023-11-08 15:40:44,090: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-11-08 15:40:51,564: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64

|